### Talk and ask about SQL
So far we demonstrated the of Open AI to help answer questions over the document, chat over the document and summarize the large document.  The focus so far was primarily on the unstructured nature of the documents.   Now we want to start looking into asking business questions by performing advanced data analytic tasks on a business database. Examples of questions are:

- Simple: Which Shipper can ship the order
- More difficult: Display Product by Category
- Advanced: Number of units in stock by category and supplier continent

For the notebook we will focus on two different applications:
- SQL Query Writing Assistant: a simple application that translate business question into SQL query language then execute and display result.
- Data Analysis Assistant: a more sophisticated application to perform advanced data analytics such as statisical analysis and forecasting. Here we demonstrate the use of [Chain of Thought](https://arxiv.org/abs/2201.11903) and [React](https://arxiv.org/abs/2210.03629) techniques to perform multi-step processing where the next step in the chain also depends on the observation/result from the previous step.


#### Pre-requisite

Deploy SQL server and [Create a empty SQL Database](https://learn.microsoft.com/en-us/azure/azure-sql/database/single-database-create-quickstart?view=azuresql&tabs=azure-portal).  Once the SQL Database is created, run [Northwind SQL](../Deployment/northwind.sql) script to create the Northwind database

Also ensure that you have the following environment variables created and configured:
- SynapseName: Synapse workspace name
- SynapsePool: Synapse pool name
- SynapseUser: Synapse user name
- SynapsePassword: Synapse password

Microsoft ODBC driver

#### Set Environment Variables

In [1]:
import os  
import json  
import openai
from Utilities.envVars import *

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"https://{OpenAiService}.openai.azure.com"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci

In [2]:
# Import the needed Python packages
import urllib
from langchain.llms.openai import AzureOpenAI, OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.schema import AgentAction

from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
    openai.api_type = "azure"
    openai.api_key = OpenAiKey
    openai.api_version = OpenAiVersion
    openai.api_base = f"https://{OpenAiService}.openai.azure.com"

    llm = AzureOpenAI(deployment_name=OpenAiDavinci,
            temperature=temperature,
            openai_api_key=OpenAiKey,
            max_tokens=tokenLength,
            batch_size=10, 
            max_retries=12)

    logging.info("LLM Setup done")
    embeddings = OpenAIEmbeddings(model=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
elif embeddingModelType == "openai":
    openai.api_type = "open_ai"
    openai.api_base = "https://api.openai.com/v1"
    openai.api_version = '2020-11-07' 
    openai.api_key = OpenAiApiKey
    llm = OpenAI(temperature=temperature,
            openai_api_key=OpenAiApiKey,
            max_tokens=tokenLength)
    embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)


This example demonstrates the use of the `SQLDatabaseChain` for answering questions over a database.

Under the hood, LangChain uses SQLAlchemy to connect to SQL databases. The SQLDatabaseChain can therefore be used with any SQL dialect supported by SQLAlchemy, such as MS SQL, MySQL, MariaDB, PostgreSQL, Oracle SQL, Databricks and SQLite. 

NOTE: For data-sensitive projects, you can specify return_direct=True in the SQLDatabaseChain initialization to directly return the output of the SQL query without any additional formatting. This prevents the LLM from seeing any contents within the database. Note, however, the LLM still has access to the database scheme (i.e. dialect, table and key names) by default.


In [3]:
synapseConnectionString = "Driver={{ODBC Driver 17 for SQL Server}};Server=tcp:{};" \
                "Database={};Uid={};Pwd={};Encrypt=yes;TrustServerCertificate=no;" \
                "Connection Timeout=30;".format(SynapseName, SynapsePool, SynapseUser, SynapsePassword)
params = urllib.parse.quote_plus(synapseConnectionString)
sqlConnectionString = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
db = SQLDatabase.from_uri(sqlConnectionString)

The concept here is using Agents as Toolkits.  In the scenario agent is applied to a specific use-case of interacting with SQL database

In [4]:
# Let's create the custom prompt that we want to use for our SQL Agent
SqlPrefix = """You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
        Always limit your query to at most {top_k} results using the SELECT TOP in SQL Server syntax.
        You can order the results by a relevant column to return the most interesting examples in the database.
        Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.
        If you get a "no such table" error, rewrite your query by using the table in quotes.
        DO NOT use a column name that does not exist in the table.
        You have access to tools for interacting with the database.
        Only use the below tools. Only use the information returned by the below tools to construct your final answer.
        You MUST double check your query before executing it. If you get an error while executing a query, rewrite a different query and try again.
        Observations from the database should be in the form of a JSON with following keys: "column_name", "column_value"
        DO NOT try to execute the query more than three times.
        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
        If the question does not seem related to the database, just return "I don't know" as the answer.
        If you cannot find a way to answer the question, just return the best answer you can find after trying at least three times."""

SqlSuffix = """Begin!
    Question: {input}
    Thought: I should look at the tables in the database to see what I can query.
    {agent_scratchpad}"""

In [5]:
topK = 3
question = "Which Shipper can ship the product?"

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agentExecutor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        prefix=SqlPrefix, 
        #suffix=SqlSuffix,
        top_k=topK,
        kwargs={"return_intermediate_steps": True}
    )
agentExecutor.return_intermediate_steps = True

logging.info("Agent Setup done")
answer = agentExecutor._call({"input":question})
intermediateSteps = answer['intermediate_steps']


Observation: CustomerDemographics, Employees, Suppliers, Shippers, Territories, OrderDetails, Categories, sysdiagrams, Customers, Products, EmployeeTerritories, CustomerCustomerDemo, Region, Orders
Thought:
Observation: 
CREATE TABLE [Shippers] (
	[ShipperID] INTEGER NOT NULL IDENTITY(1,1), 
	[CompanyName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Phone] NVARCHAR(24) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_Shippers] PRIMARY KEY ([ShipperID])
)

/*
3 rows from Shippers table:
ShipperID	CompanyName	Phone
1	Speedy Express	(503) 555-9831
2	United Package	(503) 555-3199
3	Federal Shipping	(503) 555-9931
*/


CREATE TABLE [Products] (
	[ProductID] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[SupplierID] INTEGER NULL, 
	[CategoryID] INTEGER NULL, 
	[QuantityPerUnit] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[UnitPrice] MONEY NULL DEFAULT ((0)), 
	[UnitsInStock] SMALLINT 

In [6]:
observation = ''
for item in intermediateSteps:
    agentAction: AgentAction = item[0]
    if (agentAction.tool == 'query_sql_db'):
        toolInput = str(agentAction.tool_input)
        observation = item[1]

print("Answer : " + answer['output'])
print("Observation : " + observation)
print("SQL Query : " + toolInput)

Answer : Speedy Express
Observation : [('Speedy Express',)]
SQL Query : SELECT TOP 3 Shippers.CompanyName FROM Shippers, Products WHERE Shippers.ShipperID = Products.ProductID AND Products.ProductName = 'Chai'


We covered the concept of SQLAgent, let's now look at the implementation of the SQLAgent (that uses SQLDatabaseChain) and how it can be used to interact with SQL database.

In [9]:
from langchain.chains import SQLDatabaseSequentialChain
sqlDbChainTemplate = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
    Use the following format:

    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer: "Final answer here"

    Only use the following tables:

    {table_info}
    
    Question: {input}

    """
SqlDbChainPrompt = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=sqlDbChainTemplate
)

# SqlDbChain = SQLDatabaseChain(llm=llm, database=db, prompt=SqlPrompt, verbose=True, return_intermediate_steps=True,
#                               top_k=topK)
SqlDbChain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True, 
                                                    query_prompt=SqlDbChainPrompt, top_k=topK)
question = "Display Product by Category"
answer = SqlDbChain(question)
intermediateSteps = answer['intermediate_steps']
sqlAnswer = answer['result']
toolInput = intermediateSteps[0]
observation = intermediateSteps[1]
print("Answer : " + sqlAnswer)
print("toolInput : " + str(toolInput))
print("observation : " + str(observation))




> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Categories', 'Products']

> Entering new SQLDatabaseChain chain...
Display Product by Category
SQLQuery:SELECT [ProductName], [CategoryName]
FROM [Products]
INNER JOIN [Categories]
ON [Products].[CategoryID] = [Categories].[CategoryID]
SQLResult: [('Chai', 'Beverages'), ('Chang', 'Beverages'), ('Aniseed Syrup', 'Condiments'), ("Chef Anton's Cajun Seasoning", 'Condiments'), ("Chef Anton's Gumbo Mix", 'Condiments'), ("Grandma's Boysenberry Spread", 'Condiments'), ("Uncle Bob's Organic Dried Pears", 'Produce'), ('Northwoods Cranberry Sauce', 'Condiments'), ('Mishi Kobe Niku', 'Meat/Poultry'), ('Ikura', 'Seafood'), ('Queso Cabrales', 'Dairy Products'), ('Queso Manchego La Pastora', 'Dairy Products'), ('Konbu', 'Seafood'), ('Tofu', 'Produce'), ('Genen Shouyu', 'Condiments'), ('Pavlova', 'Confections'), ('Alice Mutton', 'Meat/Poultry'), ('Carnarvon Tigers', 'Seafood'), ('Teatime Chocolate Biscuits', 'Confections'), ("

#### Test using GPT 3.5 turbo and with custom code & Prompt outside of Langchain

In [7]:
from sqlalchemy import create_engine
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import time
import re

def executeQuery(query, limit=10000):  
    engine = create_engine(sqlConnectionString)
    result = pd.read_sql_query(query, engine)
    result = result.infer_objects()
    for col in result.columns:  
        if 'date' in col.lower():  
            result[col] = pd.to_datetime(result[col], errors="ignore")  

    if limit is not None:  
        result = result.head(limit)  # limit to save memory  
    # session.close()  
    return result

In [8]:
def getTableSchema():
    sqlQuery = """  
    SELECT C.TABLE_NAME, C.COLUMN_NAME, C.DATA_TYPE, T.TABLE_TYPE, T.TABLE_SCHEMA  
    FROM INFORMATION_SCHEMA.COLUMNS C  
    JOIN INFORMATION_SCHEMA.TABLES T ON C.TABLE_NAME = T.TABLE_NAME AND C.TABLE_SCHEMA = T.TABLE_SCHEMA  
    WHERE T.TABLE_TYPE = 'BASE TABLE'  
    """  
    
    # Execute the SQL query and store the results in a DataFrame  
    df = executeQuery(sqlQuery, limit=None)  
    output=[]
    # Initialize variables to store table and column information  
    curTable = ''  
    columns = []  
    
    # Loop through the query results and output the table and column information  
    for index, row in df.iterrows():
        tableName = f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}" 
        colName = row['COLUMN_NAME']  
        dataType = row['DATA_TYPE']   
        if " " in tableName:
            tableName= f"[{tableName}]" 
        colName = row['COLUMN_NAME']  
        if " " in colName:
            colName= f"[{colName}]" 

        # If the table name has changed, output the previous table's information  
        if curTable != tableName and curTable != '':  
            output.append(f"table: {curTable}, columns: {', '.join(columns)}")  
            columns = []  
        
        # Add the current column information to the list of columns for the current table  
        columns.append(f"{colName} {dataType}")  
        
        # Update the current table name  
        curTable = tableName  
    
    # Output the last table's information  
    output.append(f"table: {curTable}, columns: {', '.join(columns)}")
    output = "\n ".join(output)
    return output

In [41]:
def callLlm(embeddingModelType, prompt, stop):
    if (embeddingModelType == 'azureopenai'):
        openai.api_type = "azure"
        openai.api_key = OpenAiKey
        openai.api_version = OpenAiVersion
        openai.api_base = f"https://{OpenAiService}.openai.azure.com"
        response = openai.ChatCompletion.create(
            engine=OpenAiChat, 
            messages = prompt,
            temperature=temperature,
            max_tokens=tokenLength,
            stop=stop
            )
        logging.info("LLM Setup done")
        llmOutput = response['choices'][0]['message']['content']
    elif embeddingModelType == "openai":
        openai.api_type = "open_ai"
        openai.api_base = "https://api.openai.com/v1"
        openai.api_version = '2020-11-07' 
        openai.api_key = OpenAiApiKey
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", 
            messages = prompt,
            temperature=temperature,
            max_tokens=tokenLength,
            stop=stop
            )
        llmOutput = response['choices'][0]['message']['content']
    return llmOutput

In [42]:
def extractOutput(textInput, extractPattern):
    output={}
    if len(textInput)==0:
        return output
    for pattern in extractPattern: 
        if "sql" in pattern[1]:

            sql_query=""
            sql_result = re.findall(pattern[1], textInput, re.DOTALL)

            if len(sql_result)>0:
                sql_query=sql_result[0]
                output[pattern[0]]= sql_query
            else:
                return output
            text_before = textInput.split(sql_query)[0].strip("\n").strip("```sql").strip("\n")

            if text_before is not None and len(text_before)>0:
                output["text_before"]=text_before
            text_after =textInput.split(sql_query)[1].strip("\n").strip("```")
            if text_after is not None and len(text_after)>0:
                output["text_after"]=text_after
            return output

        if "python" in pattern[1]:
            result = re.findall(pattern[1], textInput, re.DOTALL)
            if len(result)>0:
                output[pattern[0]]= result[0]
        else:

            result = re.search(pattern[1], textInput,re.DOTALL)
            if result:  
                output[result.group(1)]= result.group(2)

    return output

In [43]:
def getNextSteps(extractPattern, embeddingModelType, updatedUserContent, chatHistory, stop):
    oldUserContent=""
    if len(chatHistory)>1:
        oldUserContent= chatHistory.pop() #removing old history
        oldUserContent=oldUserContent['content']+"\n"
    chatHistory.append({"role": "user", "content": oldUserContent+updatedUserContent})
    #print("prompt input ", chatHistory)
    n=0
    try:
        llmOutput = callLlm(embeddingModelType, chatHistory, stop)
        #print("llmOutput \n", llmOutput)
    except Exception as e:
        time.sleep(8) #sleep for 8 seconds
        while n<5:
            try:
                llmOutput = callLlm(embeddingModelType, chatHistory, stop)
            except Exception as e:
                n +=1
                print("error calling open AI, I am retrying 5 attempts , attempt ", n)
                time.sleep(8) #sleep for 8 seconds
                print(e)

        llmOutput = "OPENAI_ERROR"
        print("llmOutput: ", llmOutput)
    output = extractOutput(llmOutput, extractPattern)
    if len(output)==0 and llmOutput != "OPENAI_ERROR": #wrong output format
        llmOutput = "WRONG_OUTPUT_FORMAT"
    return llmOutput,output

#### For first use-case test it as SQL Query Writing Assistant

In [12]:
sysDefaultMessage ="""
    You are an agent designed to interact with a SQL database with schema detail in <<data_sources>>.
    Given an input question, create a syntactically correct {sql_engine} query to run, then look at the results of the query and return the answer.
    You can order the results by a relevant column to return the most interesting examples in the database.
    Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.
    You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
    Remember to format SQL query as in ```sql\n SQL QUERY HERE ``` in your response.
    
    """
fewShotExamples=""
sqlEngine = 'sqlserver'
extractPattern=[('sql',r"```sql\n(.*?)```")]

# get Table Schema
tableSchema  = getTableSchema()
sysMessage = f"""
        <<data_sources>>
        {tableSchema}
        {sysDefaultMessage.format(sql_engine=sqlEngine)}
        {fewShotExamples}
        """
tokenLength=1000
temperature=0
embeddingModelType = "azureopenai"

In [13]:
def getAnswer(newInput, chatHistory):
    maxSteps=15
    count=1
    while count<= maxSteps:
        llmOutput,nextSteps = getNextSteps(extractPattern, embeddingModelType, newInput, chatHistory, stop=["Observation:", f"Thought {count+1}"])
        if llmOutput=='OPENAI_ERROR':
            print("Error Calling Azure Open AI, probably due to max service limit, please try again")
            break
        elif llmOutput=='WRONG_OUTPUT_FORMAT': #just have open AI try again till the right output comes
            count +=1
            continue
        output =None
        error= False

        newInput += f"\n{llmOutput}"
        for key, value in nextSteps.items():
            newInput += f"\n{value}"
            
            if "SQL" in key.upper():
                print("SQL Code : ")
                print(value)
                try:
                    output = executeQuery(value)
                except Exception as e:
                    newInput +="Encounter following error, can you try again?\n"+str(e)
                    error=str(e)
            else:
                print("Non-SQL Value : ")
                print(value)
        print("Prompt & Chat History : ")
        print(chatHistory)

        if output is not None:
            print("Output : ")
            print(output)
            break

        if error:
            print(error)

        count +=1
        if count>= maxSteps:
            print("Cannot handle the question, please change the question and try again")

In [14]:
chatHistory =  [{"role": "system", "content": sysMessage}]
question = "Which Shipper can ship the product?"
newInput= f"Question: {question}"
getAnswer(newInput, chatHistory)


SQL Code : 
SELECT s.CompanyName, s.Phone
FROM Shippers s
JOIN Products p ON s.ShipperID = p.SupplierID
WHERE p.ProductName = 'product_name';

Non-SQL Value : 
To answer this question, we need to know the available shippers in the database and their contact information. We also need to know the products available in the database and their corresponding supplier. Assuming we have this information, we can join the `Shippers` and `Products` tables on the `ShipperID` and `SupplierID` columns respectively, and filter by the desired product. Here's the SQL query:
Non-SQL Value : 


Replace `product_name` with the name of the desired product. This query will return the name and phone number of the shipper that can ship the desired product.
Prompt & Chat History : 
[{'role': 'system', 'content': '\n        <<data_sources>>\n        table: dbo.sysdiagrams, columns: name nvarchar, principal_id int, diagram_id int, version int, definition varbinary\n table: dbo.Employees, columns: EmployeeID int,

In [15]:
chatHistory =  [{"role": "system", "content": sysMessage}]
question = "Number of units in stock by category and supplier continent"
newInput= f"Question: {question}"
getAnswer(newInput, chatHistory)

SQL Code : 
SELECT c.CategoryName, s.Country, SUM(p.UnitsInStock) AS TotalUnitsInStock
FROM dbo.Products p
JOIN dbo.Suppliers s ON p.SupplierID = s.SupplierID
JOIN dbo.Categories c ON p.CategoryID = c.CategoryID
GROUP BY c.CategoryName, s.Country
ORDER BY TotalUnitsInStock DESC

Non-SQL Value : 
 
This query joins the Products, Suppliers, and Categories tables to get the category name, supplier country, and total units in stock for each combination of category and supplier continent. The results are grouped by category and supplier country and ordered by total units in stock in descending order.
Prompt & Chat History : 
[{'role': 'system', 'content': '\n        <<data_sources>>\n        table: dbo.sysdiagrams, columns: name nvarchar, principal_id int, diagram_id int, version int, definition varbinary\n table: dbo.Employees, columns: EmployeeID int, LastName nvarchar, FirstName nvarchar, Title nvarchar, TitleOfCourtesy nvarchar, BirthDate datetime, HireDate datetime, Address nvarchar, C

#### This time test it as Data Analyst Assistant

In [44]:
daSysDefaultMessage = """
        You are a smart AI assistant to help answer business questions based on analyzing data. 
        You can plan solving the question with one more multiple thought step. At each thought step, you can write python code to analyze data to assist you. Observe what you get at each step to plan for the next step.
        You are given following utilities to help you retrieve data and commmunicate your result to end user.
        1. executeQuery(sql_query: str): A Python function can query data from the <<data_sources>> given a query which you need to create. The query has to be syntactically correct for {sql_engine} and only use tables and columns under <<data_sources>>. The executeQuery function returns a Python pandas dataframe contain the results of the query.
        2. Use plotly library for data visualization. 
        3. Use observe(label: str, data: any) utility function to observe data under the label for your evaluation. Use observe() function instead of print() as this is executed in streamlit environment. Due to system limitation, you will only see the first 10 rows of the dataset.
        4. To communicate with user, use show() function on data, text and plotly figure. show() is a utility function that can render different types of data to end user. Remember, you don't see data with show(), only user does. You see data with observe()
            - If you want to show  user a plotly visualization, then use ```show(fig)`` 
            - If you want to show user data which is a text or a pandas dataframe or a list, use ```show(data)```
            - Never use print(). User don't see anything with print()
        5. Lastly, don't forget to deal with data quality problem. You should apply data imputation technique to deal with missing data or NAN data.
        6. Always follow the flow of Thought: , Observation:, Action: and Answer: as in template below strictly. 

        """
daFewShotExamples ="""
        <<Template>>
        Question: User Question
        Thought 1: Your thought here.
        Action: 
        ```python
        #Import neccessary libraries here
        import numpy as np
        #Query some data 
        sql_query = "SOME SQL QUERY"
        step1_df = executeQuery(sql_query)
        # Replace NAN with 0. Always have this step
        step1_df['Some_Column'] = step1_df['Some_Column'].replace(np.nan,0)
        #observe query result
        observe("some_label", step1_df) #Always use observe() instead of print
        ```
        Observation: 
        step1_df is displayed here
        Thought 2: Your thought here
        Action:  
        ```python
        import plotly.express as px 
        #from step1_df, perform some data analysis action to produce step2_df
        #To see the data for yourself the only way is to use observe()
        observe("some_label", step2_df) #Always use observe() 
        #Decide to show it to user.
        fig=px.line(step2_df)
        #visualize fig object to user.  
        show(fig)
        #you can also directly display tabular or text data to end user.
        show(step2_df)
        ```
        Observation: 
        step2_df is displayed here
        Answer: Your final answer and comment for the question. Also use Python for computation, never compute result youself.
        <</Template>>

        """
sqlEngine = 'sqlserver'
daExtractPattern=[("Thought:",r'(Thought \d+):\s*(.*?)(?:\n|$)'), ('Action:',r"```python\n(.*?)```"),("Answer:",r'([Aa]nswer:) (.*)')]

# get Table Schema
tableSchema  = getTableSchema()
daSysMessage = f"""
        <<data_sources>>
        {tableSchema}
        {daSysDefaultMessage.format(sql_engine=sqlEngine)}
        {daFewShotExamples}
        """
tokenLength=1000
temperature=0
embeddingModelType = "openai"


In [45]:
def getDaAnswer(extractPattern, newInput, chatHistory):

    from plotly.graph_objects import Figure
    import numpy as np
    import plotly.express as px
    import plotly.graph_objs as go
    import pandas as pd

    def show(data):
        #if type(data) is Figure:
        #    plotly_chart(data)
        #else:
        print(data)
        if type(data) is not Figure:
            print('observation: this was shown to user',data)

    def observe(name, data):
        try:
            data = data[:10] # limit the print out observation to 15 rows
        except:
            pass
        print(f'observation:{name}', data)
    
    def executeQuery(query, limit=10000):  
        engine = create_engine(sqlConnectionString)
        result = pd.read_sql_query(query, engine)
        result = result.infer_objects()
        for col in result.columns:  
            if 'date' in col.lower():  
                result[col] = pd.to_datetime(result[col], errors="ignore")  

        if limit is not None:  
            result = result.head(limit)  # limit to save memory  
        # session.close()  
        return result
        
    maxSteps=15
    count=1
    finish = False
    while not finish:
        llmOutput,nextSteps = getNextSteps(extractPattern, embeddingModelType, newInput, chatHistory, stop=["Observation:", f"Thought {count+1}"])
        if llmOutput=='OPENAI_ERROR':
            print("Error Calling Azure Open AI, probably due to max service limit, please try again")
            break
        elif llmOutput=='WRONG_OUTPUT_FORMAT': #just have open AI try again till the right output comes
            count +=1
            continue

        newInput += f"\n{llmOutput}"
        for key, value in nextSteps.items():
            newInput += f"\n{value}"
            
            if "ACTION" in key.upper():
                print("SQL Code : ")
                print(key)
                print(value)
                observations =[]
                serialized_obs=[]
                try:
                    exec(value, locals())
                    if "observation:" in key:
                        observations.append((key.split(":")[1],observation))
                        if type(observation) is pd:
                            serialized_obs.append((key.split(":")[1],observation.to_string()))
                        elif type(observation) is not Figure:
                            serialized_obs.append({key.split(":")[1]:str(observation)})
                except Exception as e:
                    observations.append(("Error:",str(e)))
                    serialized_obs.append({"\nEncounter following error, can you try again?\n:":str(e)+"\nAction:"})
                
                for observation in observations:
                    print(observation[0])
                    print(observation[1])
                obs = f"\nObservation on the first 10 rows of data: {serialized_obs}"
                newInput += obs
            else:
                print(key)
                print(value)
            if "Answer" in key:
                print("Answer is given, finish")
                finish= True
        print("Prompt & Chat History : ")
        print(chatHistory)

        count +=1
        if count>= maxSteps:
            print("Cannot handle the question, please change the question and try again")

In [46]:
daChatHistory =  [{"role": "system", "content": daSysMessage}]
#daQuestion = "Number of units in stock by category and supplier continent"
daQuestion = "Display Product by Category"
daNewInput= f"Question: {daQuestion}"
getDaAnswer(daExtractPattern, daNewInput, daChatHistory)

Thought 1
We need to join the Products and Categories table to get the Product by Category. 
SQL Code : 
Action:
#Import neccessary libraries here
import numpy as np
import plotly.express as px

#Query some data 
sql_query = "SELECT p.ProductName, c.CategoryName FROM dbo.Products p JOIN dbo.Categories c ON p.CategoryID = c.CategoryID"
product_by_category_df = executeQuery(sql_query)

# Replace NAN with 0. Always have this step
product_by_category_df = product_by_category_df.replace(np.nan,0)

#observe query result
observe("Product by Category", product_by_category_df) #Always use observe() instead of print

observation:Product by Category                        ProductName  CategoryName
0                             Chai     Beverages
1                            Chang     Beverages
2                    Aniseed Syrup    Condiments
3     Chef Anton's Cajun Seasoning    Condiments
4           Chef Anton's Gumbo Mix    Condiments
5     Grandma's Boysenberry Spread    Condiments
6  Uncle B